In [1]:

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import SQLContext
import pymongo
import pandas as pd
from pyspark import SparkFiles


In [2]:
conf = SparkConf().set('spark.driver.host','127.0.0.1')
sc = SparkContext(master='local', appName='myAppName',conf=conf)

In [3]:
spark = SparkSession.builder.master("local[1]").appName("test").getOrCreate()

In [4]:
client = pymongo.MongoClient()

In [5]:
mydb = client['covid']
mycol = mydb["timeseries"]
mycol1 = mydb["all_records"]

In [6]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" # Make sure the url is the raw version of the file on GitHub

In [7]:
url1 = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv"

In [8]:
sc.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("time_series_covid19_confirmed_global.csv"), header=True, inferSchema= True)

In [9]:
df = df.drop(col('Province/State'))
r = df.toPandas()

In [10]:
all_r = r.to_dict('records')
mycol1.insert_many(all_r)

In [11]:
sc.addFile(url1)
df1 = spark.read.csv("file://"+SparkFiles.get("owid-covid-latest.csv"), header=True, inferSchema= True)

In [12]:
newdf = df1.toPandas()

In [13]:
newdf = newdf[['location','total_cases','total_deaths']]

In [14]:
mycol.insert_many(newdf.to_dict('records'))

In [15]:
# cols = df.columns[3:]

# def column_add(a,b):
#      return  a.__add__(b)
# newdf = df.withColumn('total', reduce(column_add, ( df[col] for col in cols ) ))

# r = newdf.toPandas()

# r.tail()

# r = r.rename(columns={'Country/Region':'Country'})

# newdf1 = r[['Country','Lat','Long','total']]

# # all_r = dict.fromkeys(r.columns)



# to_mongo = []
# for row in newdf1.itertuples():
#     to_mongo.append({'Country':row.Country,'Lat':row.Lat,'Long':row.Long,'Total':row.total})

In [16]:
r

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,11/11/21,11/12/21,11/13/21,11/14/21,11/15/21,11/16/21,11/17/21,11/18/21,11/19/21,11/20/21
0,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,...,156456,156487,156510,156552,156610,156649,156739,156739,156812,156864
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,191440,192013,192600,193075,193269,193856,194472,195021,195523,195988
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,207624,207764,207873,207970,208104,208245,208380,208532,208695,208839
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,15744,15819,15819,15819,15907,15929,15972,16035,16086,16086
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,64857,64875,64899,64913,64913,64940,64968,64985,64997,65011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,...,1000897,1009879,1018346,1026522,1035138,1045397,1055246,1065469,1075094,1084625
276,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,...,456407,456632,456632,456632,457154,457390,457477,457729,457950,457950
277,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,9907,9912,9918,9918,9936,9936,9947,9950,9954,9955
278,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,...,209953,209963,209971,209983,209996,210008,210020,210036,210043,210057
